### This is the definitive version for producing CMIP5 time series

In [2]:
import numpy as np
import pandas as pd
import xarray as xy
import string
from copy import deepcopy
import os.path
import matplotlib.pyplot as plt
from time import time

from constants import * #<= my defined global variables

%matplotlib inline

In [3]:
def getFileInfo(dataDir,fileNameSplitter='_',fileNamePos=2):
    def justDataFiles(path=None):
        for f in os.listdir(path):
            if ('p1_21' not in f) and ('p1_22' not in f) and ('.nc' in f):
                yield f
            
    files = [x for x in justDataFiles(dataDir)]  
    files.sort()
    
    #make a list of the model names in this file list
    modelNames = []
    for oneFile in files:
        justFileName = str.split(oneFile,'/')[-1]
        #print justFileName
        justEnsMemNum = str.split(justFileName,fileNameSplitter)[fileNamePos]
        #print justEnsMemNum
        modelNames.append(justEnsMemNum)
    modelNames = list(set(modelNames))

    #make a dictionary of all the files that go with each model
    keyvaluepairs = []
    for model in modelNames:
        uniqueModelName = ''.join([model,fileNameSplitter])
        oneModelFileList = []
        for oneFile in files:
            if uniqueModelName in oneFile:
                oneModelFileList.append(oneFile)
        tupleOfKeyValue = (model,oneModelFileList)
        keyvaluepairs.append(tupleOfKeyValue)

    fileDict = dict(keyvaluepairs)
    
    return fileDict

In [4]:
def getAreaAvg(data,bounds,weights,landFrac): #,t,weights):

    weightedMean = 0.0
    weightTotal = 0.0
    
    areaSubset = data.sel(lon=slice(bounds.lonMin,bounds.lonMax),lat=slice(bounds.latMin,bounds.latMax)) #.isel(time=t)
    if landFrac and excludeOcean:
        landFrac = landFrac.sel(lon=slice(bounds.lonMin,bounds.lonMax),lat=slice(bounds.latMin,bounds.latMax))
        areaSubset = areaSubset.where(landFrac>0.5)
        
    useCosLat = False
    if weights is None:
        useCosLat = True
    else:
        try:
            if len(weights.dims)==2:
                weightsSubset = weights.sel(lon=slice(bounds.lonMin,bounds.lonMax),lat=slice(bounds.latMin,bounds.latMax))
                weightedMean = (areaSubset * np.asarray(weightsSubset)).sum('lat').sum('lon')
            elif len(weights.dims)==1:
                weightsSubset = weights.sel(lat=slice(bounds.latMin,bounds.latMax))
                weightedMean = (areaSubset.mean('lon') * np.asarray(weightsSubset)).sum('lat')
        except:
            #try *not* using the weights from the files, which might need to be updated
            useCosLat = True
            print "an exception!", weights.shape
            
    if useCosLat:
        weightsSubset = np.cos(np.deg2rad(areaSubset.lat))
        weightedMean = (areaSubset.mean('lon') * np.asarray(weightsSubset)).sum('lat')
        
    weightTotal = weightsSubset.sum()
    return weightedMean/np.double(weightTotal)

def convertPrecipUnits(dataVsTime, startingFrom=''): # converts to mm/day                                                                                                                       
    if startingFrom == 'm/s':
        return dataVsTime * 1000.0 * 60.0 * 60.0 * 24.0
    else: #default for CMIP5                                                                                                                           
        return dataVsTime * 60.0*60.0*24.0

In [5]:
def getOneFile(filename,field,bounds,weightFileName,landFracFileName):
    
    oneEnsMem = xy.open_dataset(filename)
        
    #get weights from weighta file if exists:
    try:
        weights = xy.open_dataset(weightFileName)
        weights = weights.areacella
    except:
        weights = None
        
    if excludeOcean:
        #get land fraction data if exists:
        try:
            landFrac = xy.open_dataset(landFracFileName)
            landFrac = landFrac.sftlf
        except:
            landFrac = None
    else:
        landFrac = None
            
    #we're not looking out past 2100, which a few models provide, and anyway it would cause indexing errors
    if ('historical' not in filename) and ('CESMens' not in filename):
        if ('IPSL-CM5A-LR' in filename or 'CSIRO-Mk3L-1-2' in filename): #len(oneEnsMem.time) >= 2262:
            oneEnsMem = oneEnsMem.isel(time=slice(0,1140)) #just until 2100-12 century
            oneEnsMem['time'] = pd.date_range('2006-01-01',periods=1140,freq='MS')
        oneEnsMem = oneEnsMem.sel(time=slice('2006-01-01','2099-12-31')) #a bit more exactly
        #if(np.bool(oneEnsMem.time[-1] < np.datetime64('2099-11-30'))):
            #print "file doesn't go to end-of-century, only ", np.datetime_as_string(oneEnsMem.time[-1])
    
    timeSeries = getAreaAvg(oneEnsMem[field],bounds,weights,landFrac).to_series() 
    if field == 'tas':
        if oneEnsMem[field].units != 'K':
            print filename
            print oneEnsMem[field].units
        if np.double(oneEnsMem[field].mean()) < 200.0:
            print filename
            print oneEnsMem[field].units
    
    if field == 'pr':
        timeSeries = convertPrecipUnits(timeSeries)
    
    try:    
        timeSeries.index = timeSeries.index.to_period(freq='M') 
    except:
        print filename, " has dates out of range"
        print timeSeries.index
    
    return timeSeries

In [6]:
def getOneModelAllFiles(dataDir,fileList,field,bounds,fileNameSplitter='_',fileNamePos=2,ensMemNumPos=4):
    justFileName = fileList[0]
    justModelName = str.split(justFileName,fileNameSplitter)[fileNamePos]
    weightFileName = ''.join([weightPathAndPrefix,justModelName,weightSuffix]) #see constants.py
    landFracFileName = None
    if excludeOcean:
        try:
            landFracFileName = landFracFiles[justModelName][0]
        except:
            print "no land frac file for ", justModelName
    
    ensMemList = []
    for filename in fileList:
        ensMemNum = str.split(filename,fileNameSplitter)[ensMemNumPos]
        ensMemNum = str.split(ensMemNum,'i')[0]
        ensMemNum = np.int(str.split(ensMemNum,'r')[1]) #the one or two digit integer between r and i in _r?i?p?_
        if ensMemNum not in ensMemList:
            ensMemList.append(ensMemNum)
    
    ensMemList.sort()
    #print ensMemList
    
    def getOneEnsembleMember(ensMem):
        firstFile = True
        for filename in fileList:
            ensMemNum = str.split(filename,fileNameSplitter)[ensMemNumPos]
            ensMemNum = str.split(ensMemNum,'i')[0]
            ensMemNum = np.int(str.split(ensMemNum,'r')[1]) #the one or two digit integer between r and i in _r?i?p?_

            if ensMemNum == ensMem:
                #print filename
                onePartOfTimeSeries = getOneFile(''.join([dataDir,filename]),field,bounds,weightFileName,landFracFileName)
                onePartOfTimeSeries.name = ensMemNum
                if firstFile:
                    timeSeries = onePartOfTimeSeries
                    firstFile = False
                else:
                    timeSeries = pd.concat([timeSeries,onePartOfTimeSeries],axis=0)

        timeSeries.name = ''.join(['run',str(ensMem)])
        #print timeSeries.name
        #--------------
        to_return = pd.DataFrame(timeSeries)
    
        #This last bit is necessary because at least one file has duplicate rows, probably from ncrcatting together
        #more than one version of the same file. The values in the duplicate rows are not all identical.
        to_return = to_return.reset_index().drop_duplicates(subset='time',keep='last').set_index('time')
        to_return = to_return.sort_index()

        return to_return.transpose()
    
    timeSeriesModel = pd.concat([getOneEnsembleMember(ensMem) for ensMem in ensMemList], axis=0)
    timeSeriesModel.name = justModelName
        
    return timeSeriesModel.transpose()


## Do all regions, using pr or tas data

In [7]:
scenariosCMIP = ['rcp85','rcp45','rcp60','historical','rcp26']
fields = ['pr','tas']
excludeOcean = True

landFracFiles = getFileInfo(landFracPath)

for scenario in scenariosCMIP:
    for field in fields:
        # edit dataDir to reflect your directory structure
        dataDir = ''.join([dataDirCMIP,scenario,'/',field,'/'])
        fileDict = getFileInfo(dataDir)
        if excludeOcean:
            # leave if off if there is no land-sea mask:
            for key in fileDict.keys():
                if key not in landFracFiles.keys():
                    del fileDict[key]
        
        # regionBounds is a dictionary defined as a global variable in constants.py
        for regionKey in regionBounds:
            if regionKey != 'global':
                excludeOcean = True
            else:
                excludeOcean = False
            print scenario, field, regionKey
            time0 = time()
            allData = pd.DataFrame()
            allData = pd.concat([getOneModelAllFiles(dataDir,fileDict[model],field,regionBounds[regionKey]).transpose() for model in fileDict.keys()],axis=0,keys=fileDict.keys())
            indexToUse = pd.MultiIndex.from_tuples(allData.transpose(),names=['model','run'])
            allData.index = indexToUse
            allData = allData.transpose()
            if scenario == 'historical':
                allData = allData['185001':'200512']
            if excludeOcean:
                outFile = ''.join(['timeSeries/timeSeriesLO_',field,'_',regionKey,'_Monthly_',scenario,'.csv'])
            else:
                outFile = ''.join(['timeSeries/timeSeries_',field,'_',regionKey,'_Monthly_',scenario,'.csv'])
            allData.to_csv(outFile)
            time1 = time()
            print ' '.join([regionKey,field,scenario,'took',str(time1-time0)])


/home/disk/grease/naomi/anaconda/lib/python2.7/site-packages/xarray/conventions.py:393: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/home/disk/grease/naomi/anaconda/lib/python2.7/site-packages/xarray/conventions.py:412: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  calendar=self.calendar)


rcp85 pr BC
BC pr rcp85 took 91.6717500687
rcp85 pr global
global pr rcp85 took 268.291644096
rcp85 pr Alaska
Alaska pr rcp85 took 62.1464021206
rcp85 pr pnw
pnw pr rcp85 took 56.437194109
rcp85 pr Cali
Cali pr rcp85 took 51.7437930107
rcp85 pr Baja
Baja pr rcp85 took 80.6927351952
rcp85 tas BC
BC tas rcp85 took 364.333054066
rcp85 tas global
global tas rcp85 took 500.049427986
rcp85 tas Alaska
Alaska tas rcp85 took 206.385894775
rcp85 tas pnw
pnw tas rcp85 took 197.864928961
rcp85 tas Cali
Cali tas rcp85 took 208.137690067
rcp85 tas Baja
Baja tas rcp85 took 230.155840874
rcp45 pr BC
BC pr rcp45 took 186.089550972
rcp45 pr global
global pr rcp45 took 319.948368073
rcp45 pr Alaska
Alaska pr rcp45 took 16.2066869736
rcp45 pr pnw
pnw pr rcp45 took 15.1619939804
rcp45 pr Cali
Cali pr rcp45 took 15.2247130871
rcp45 pr Baja
Baja pr rcp45 took 15.0078449249
rcp45 tas BC
BC tas rcp45 took 479.560211897
rcp45 tas global
global tas rcp45 took 496.964792013
rcp45 tas Alaska
Alaska tas rcp45 took 